# Introduction to atomman: Cluster generators

*added version 1.4.0*

In addition to crystal structures, atomman has tools supporting calculations involving small atomic clusters.  

## 1. BondAngleMap

__NOTE__ The methods associated with this class are still in development and testing and therefore may undergo substantial updates within the next few versions. Examples also to come later...

The BondAngleMap class generates three atom clusters to explore and map the bond angle energies associated with an interatomic potential.  This is meant to provide a means of characterizing a potential's three-body interactions. 

### 1.1. Method and theory

The phase space associated with a cluster of three atoms i, j and k is characterized using the following three coordinates:

- __$r_{ij}$__ being the radial distance between atoms i and j.
- __$r_{ik}$__ being the radial distance between atoms i and k.
- __$\theta_{ijk}$__ being the angle between the ij and ik vectors.

A map of the phase space can then be constructed by taking equal steps along these three coordinates according to sampling parameters rmin, rmax, and rnum for the two radial distances and thetamin, thetamax and thetanum for the angle.  For each configuration, the radial distance between atoms j and k can be computed using $r_{jk} = \sqrt{r_{ij}^2 + r_{ik}^2 - 2 r_{ij} r_{ik} \cos(\theta_{ijk})}$ 

Atomic cluster configurations are constructed based on the three coordinates as
- atom i is positioned at $[0.0, 0.0, 0.0]$
- atom j is positioned at $[r_{ij}, 0.0, 0.0]$
- atom k is positioned at $[r_{ik} \cos(\theta_{ijk}), r_{ik} \sin(\theta_{ijk}), 0.0] $
- box dimensions of at least $\pm$rmax in all directions and non-periodic boundary conditions.
                

### 1.2. Initializing a BondAngleMap

The cluster coordinates (r distances and theta angles) are required and can be specified in one of three ways.

- A data model containing all the information.
- Input ranges (rmin, rmax, rnum, thetamin, thetamax, thetanum).  The r
  range parameters will be used to generate values for both r_ij and
  r_ik.
- Explicitly giving r_ij, r_ik and theta values.

Parameters
        
- __model__ (*str, path-like object or DataModelDict, optional*) Collected data model results from a series of runs.  Contains both coordinate information and energy values.
- __rmin__ (*float, optional*) The minimum value used for the r_ij and r_ik spacings.
- __rmax__ (*float, optional*) The maximum value used for the r_ij and r_ik spacings.
- __rnum__ (*float, optional*) The number of values used for the r_ij and r_ik spacings.
- __thetamin__ (*float, optional*) The minimum value used for the theta angles.
- __thetamax__ (*float, optional*) The maximum value used for the theta angles.
- __thetanum__ (*float, optional*) The number of values used for the theta angles.
- __r_ij__ (*array-like, optional*) All r_ij values used.  If given, the lengths of r_ij, r_ik and theta need to be the same.
- __r_ik__ (*array-like, optional*) All r_ik values used.  If given, the lengths of r_ij, r_ik and theta need to be the same.
- __theta__ (*array-like, optional*) All theta values used.  If given, the lengths of r_ij, r_ik and theta need to be the same.
- __energy__ (*array-like, optional*) All measured energies.  If r_ij, r_ik and theta are given then all should be the same length.  If the coordinate range parameters are given, then the energies should be of length rnum*rnum*thetanum and ordered to correspond to three embedded loops with r_ij iterating in the outside loop, r_ik in the middle and theta in the inside.  If energy is not given, then all values will initially be set to np.nan.
- __symbols__ (*str or list, optional*) Element model symbol(s) to associate with the three atoms if/when systems are created.  Can either be a single symbol to assign to all atoms, or three symbols to assign to atoms i, j, and k individually.  Not needed if systems are not generated by this class.

### 1.3. Iterators 

#### 1.3.1. itercoords

Iterates through the set of r_ij, r_ik, r_jk and theta_ijk coordinates for each configuration. 

#### 1.3.2. itersystems()

Iterates through the three-body coordinates based on the r and theta values. 

Parameters
        
- __symbols__ (*str or list, optional*) The element model symbols to assign to the atoms.  Can either be one value for all atoms, or three values for each atom individually. If not given here, will use the values set during class initialization.
- __copy__ (*bool, optional*) If False (default), then the yielded system is the same object with the coordinates shifted.  If True, each yielded system is a new object.

Yields

- (*atomman.System*) The atomic system containing the three-body cluster. 

### 1.4. Data representations

#### 1.4.1. df

Generates a pandas.DataFrame of the coordinates and the associated energies, if measured.

#### 1.4.2. model()

Loads or generates a bond angle map data model.

Note: Generating data models is currently limited to regular values, i.e. ones which the coordinates correspond to embedded loops with r_ij iterating in the outside loop, r_ik in the middle loop and theta in the inside loop.

Parameters

- __model__ (*str, file-like object or DataModelDict, optional*) The data model content or file containing the bond angle map data. If given, the content will be read in and set to the current object. If not given, then a data model will be returned for the object.
- __length_unit__ (*str, optional*) The unit of length to save the rmin and max values in when generating a data model.  Default value is 'angstrom'.
- __energy_unit__ (*str, optional*) The unit of energy to save the energy values in when generating a data model.  Default value is 'eV'.

Returns

- (*DataModelDict.DataModelDict*) The data model containing the bond angle map coordinate information and measured energies.  Only returned if model is not given as a parameter.

Raises

- *(ValueError*) If the data is irregular, i.e. coordinates do not conform to embedded loops with r_ij in the outer loop, r_ik in the middle loop and theta in the inside loop.

#### 1.4.3. save_table()

Saves a tabulated representation of the coordinates and energy values to a file.

Parameters
        
- __filename__ (*str*) The path to the file where the table will be saved.
- __include_header__ (*bool*) If True (default) then header comments will be listed at the top of the file.

### 1.5 Data analysis methods

#### 1.5.1. pdf()

Returns the probability density function for the energy
        
Parameters

- __nbins__ (*int, optional*) The number of histogram bins to use.  Default value is 301.
- __energymin__ (*float, optional*) The minimum energy bound to consider.  Default value is -15.0.
- __energymax__ (*float, optional*) The maximum energy bound to consider.  Default value is 15.0.

Returns
        
- __pdf__ (*numpy.NDArray*) The probability density function associated with each bin.
- __centers__ (*numpy.NDArray*) The center values for each bin.
            
#### 1.5.2. cumulative_pdf()

Returns the cumulative probability density function for the energy.
        
Parameters
        
- __nbins__ (*int, optional*) The number of histogram bins to use.  Default value is 301.
- __energymin__ (*float, optional*) The minimum energy bound to consider.  Default value is -15.0.
- __energymax__ (*float, optional*) The maximum energy bound to consider.  Default value is 15.0.

Returns
        
- __cum_pdf__ (*numpy.NDArray*) The cumulative probability density function associated with each bin.
- __centers__ (*numpy.NDArray*) The center values for each bin.
            
#### 1.5.3. plot_pdf()

Generates a plot of the probability density function of the energy.

Parameters
        
- __nbins__ (*int, optional*) The number of histogram bins to use.  Default value is 301.
- __energymin__ (*float, optional*) The minimum energy bound to consider.  Default value is -15.0.
- __energymax__ (*float, optional*) The maximum energy bound to consider.  Default value is 15.0.
- __matplotlib_axes__ (*matplotlib.Axes.axes, optional, optional*) An existing plotting axis to add the pdf plot to.  If not given, a new figure object will be generated.
- __\*\*kwargs__ (*any, optional*) Any additional key word arguments will be passed to matplotlib.pyplot.figure for generating a new figure object (if axis is not given).
        
Returns
        
- (*matplotlib.Figure*) The generated figure.  Not returned if matplotlib_axes is given.
            
#### 1.5.4. plot_cumulative_pdf()

Generates a plot of the cumulative probability density function of the energy.

Parameters
        
- __nbins__ (*int, optional*) The number of histogram bins to use.  Default value is 301.
- __energymin__ (*float, optional*) The minimum energy bound to consider.  Default value is -15.0.
- __energymax__ (*float, optional*) The maximum energy bound to consider.  Default value is 15.0.
- __matplotlib_axes__ (*matplotlib.Axes.axes, optional, optional*) An existing plotting axis to add the pdf plot to.  If not given, a new figure object will be generated.
- __\*\*kwargs__ (*any, optional*) Any additional key word arguments will be passed to matplotlib.pyplot.figure for generating a new figure object (if axis is not given).
        
Returns
        
- (*matplotlib.Figure*) The generated figure.  Not returned if matplotlib_axes is given.